# Airline Passenger Satisfaction

What factors lead to customer satisfaction for an Airline?


## Appendex
- Cick on the links:
    >1.  <a href=#imports>Imports</a>
    >2.  <a href=#loaddata>Load Data</a>
    >3.  <a href=#dataformating>Data Formating & Feature Engineering</a>
    >4.  <a href=#Expl>Create X and y</a>
    >5.  <a href=#Ques> Dummy variables and label encoder </a>
    >6.  <a href=#d> Split the data</a>
    >7.  <a href=#b> Baseline Model </a>
    >8.  <a herf=#ss>Standerd Scale with grid search</a>
    >9.  <a herf=#ssss> Decision Tree & GridSearchCV Tune the parameters</a>
    >10. <a herf=#Rf>Random Forest with GridSearchCV Tune the parameters</a>
    >11. <a herf=#we>Soft voting </a>
    >12. <a herf=#sc>  Stacking Classifier </a>  
    >13. <a herf=#cm> Confution Matrix </a> 
    >14. <a herf=#jp>  XGboost</a>       

    

# Abstract

What factors lead to customer satisfaction for an Airline? This dataset contains an airline passenger satisfaction survey. What factors are highly correlated to a satisfied (or dissatisfied) passenger? Can you predict passenger satisfaction?

# Design

This project is one of the T5 Data Science Boot Camp requirements. Data provided by Kaggle. In this module we will be laying the foundation for our analysis by processing and exploring a large amount of data and classify it by using classification modules. The dataset contains an airline passenger satisfaction survey. 



# <a name='imports' > Import Libraries </a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set()
# modeling imports
from sklearn.metrics import accuracy_score
import pickle
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, fbeta_score,f1_score ,confusion_matrix, classification_report, make_scorer
from mlxtend.classifier import StackingClassifier # <-- note: this is not from sklearn!
from sklearn import linear_model, neighbors, ensemble
sns.set_style("whitegrid")

# <a name='loaddata' > Load Data </a>

In [3]:
df = pd.read_csv('Airline.csv')

In [4]:
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,...,2,2,2,2,4,2,4,0,20.0,satisfied


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         25976 non-null  int64  
 1   id                                 25976 non-null  int64  
 2   Gender                             25976 non-null  object 
 3   Customer Type                      25976 non-null  object 
 4   Age                                25976 non-null  int64  
 5   Type of Travel                     25976 non-null  object 
 6   Class                              25976 non-null  object 
 7   Flight Distance                    25976 non-null  int64  
 8   Inflight wifi service              25976 non-null  int64  
 9   Departure/Arrival time convenient  25976 non-null  int64  
 10  Ease of Online booking             25976 non-null  int64  
 11  Gate location                      25976 non-null  int

# <a name='dataformating' > Data Formating & Features Engineering</a>

In [6]:
df.drop(['Gate location'], axis= 'columns', inplace=True)

In [7]:
df.drop(['Unnamed: 0', 'id'], axis= 'columns', inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             25976 non-null  object 
 1   Customer Type                      25976 non-null  object 
 2   Age                                25976 non-null  int64  
 3   Type of Travel                     25976 non-null  object 
 4   Class                              25976 non-null  object 
 5   Flight Distance                    25976 non-null  int64  
 6   Inflight wifi service              25976 non-null  int64  
 7   Departure/Arrival time convenient  25976 non-null  int64  
 8   Ease of Online booking             25976 non-null  int64  
 9   Food and drink                     25976 non-null  int64  
 10  Online boarding                    25976 non-null  int64  
 11  Seat comfort                       25976 non-null  int

In [9]:
df.nunique()

Gender                                  2
Customer Type                           2
Age                                    75
Type of Travel                          2
Class                                   3
Flight Distance                      3281
Inflight wifi service                   6
Departure/Arrival time convenient       6
Ease of Online booking                  6
Food and drink                          6
Online boarding                         6
Seat comfort                            5
Inflight entertainment                  6
On-board service                        6
Leg room service                        6
Baggage handling                        5
Checkin service                         5
Inflight service                        6
Cleanliness                             6
Departure Delay in Minutes            313
Arrival Delay in Minutes              320
satisfaction                            2
dtype: int64

In [10]:
df.isna().sum()

Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             83
satisfaction                          0
dtype: int64

In [11]:
sum(df.duplicated())

0

In [12]:
df = df.dropna()

In [13]:
df.isna().sum()

Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Inflight wifi service                0
Departure/Arrival time convenient    0
Ease of Online booking               0
Food and drink                       0
Online boarding                      0
Seat comfort                         0
Inflight entertainment               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Inflight service                     0
Cleanliness                          0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
satisfaction                         0
dtype: int64

# <a name='Expl' > Creat X and Y</a>

In [14]:
X = df.drop(['satisfaction'], axis = 1)
y = df['satisfaction']

# <a name='Ques' > Dummy variables and label Encoder</a>

In [15]:
le = LabelEncoder()
y=le.fit_transform(y)

In [16]:
X= pd.get_dummies(X)

In [17]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25893 entries, 0 to 25975
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                25893 non-null  int64  
 1   Flight Distance                    25893 non-null  int64  
 2   Inflight wifi service              25893 non-null  int64  
 3   Departure/Arrival time convenient  25893 non-null  int64  
 4   Ease of Online booking             25893 non-null  int64  
 5   Food and drink                     25893 non-null  int64  
 6   Online boarding                    25893 non-null  int64  
 7   Seat comfort                       25893 non-null  int64  
 8   Inflight entertainment             25893 non-null  int64  
 9   On-board service                   25893 non-null  int64  
 10  Leg room service                   25893 non-null  int64  
 11  Baggage handling                   25893 non-null  int

In [18]:
df.shape

(25893, 22)

# <a name='d' > Split the data </a>

In [19]:
#Split the data
X_train, X_val, y_train, y_val = \
        train_test_split(X, y, test_size=0.2, random_state=42)

# <a name='b' > Baseline Model  </a>

In [20]:
 
linear_model = LogisticRegression()
linear_model.fit(X_train, y_train)
print( linear_model.score(X_train, y_train))
print(linear_model.score(X_val, y_val))

0.806749058607705
0.8074917937825835


# Pickle approach

In [82]:
# Save the Modle to file in the current working directory

Pkl_Filename = "Pickle_RL_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(linear_model, file)

In [83]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

Pickled_LR_Model

LogisticRegression()

In [85]:

# Use the Reloaded Model to 
# Calculate the accuracy score and predict target values

# Calculate the Score 
score = Pickled_LR_Model.score(X_val, y_val)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score))  

# Predict the Labels using the reloaded Model
Ypredict = Pickled_LR_Model.predict(X_val)  

Ypredict

Test score: 80.75 %


array([1, 0, 1, ..., 0, 0, 0])

# <a name='ss' > Standerd Scale with the Grid search </a>

In [21]:

std_scale= StandardScaler()
std_scale.fit(X_train)
Xtrain_scale = std_scale.transform(X_train)
Xval_scale = std_scale.transform(X_val)

linear_model.fit(Xtrain_scale, y_train)
print(linear_model.score(Xtrain_scale, y_train))
print(linear_model.score(Xval_scale ,y_val))

0.8709085642560587
0.8710175709596447


In [22]:
# Grid search 
# define the parameter values that should be searched
k_range = list(range(1, 100))
print(k_range)
# create a parameter grid: map the parameter names to the values that should be searched
param_grid = dict(n_neighbors=k_range)
print(param_grid)

# instantiate the grid
knn = KNeighborsClassifier(n_neighbors=5)
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]}


In [ ]:
# fit the grid with data
grid.fit(X, y);
# view the complete results
grid.cv_results_

# Examine the best model

In [ ]:

print("Best params: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

In [ ]:
knn_best = grid.best_estimator_

# <a name='ssss' > Decision Tree &  GridSearchCV Tune the parameters  </a>

In [ ]:

decisiontree = DecisionTreeClassifier()
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)
decisiontree.fit(xtrain, ytrain)
print(decisiontree.score(xtrain, ytrain))
print(decisiontree.score(xtest, ytest))


In [ ]:
# Save the Modle to file in the current working directory

Pk2_Filename = "Pickle_RL_Model.pk2"  

with open(Pk2_Filename, 'wb') as file:  
    pickle.dump(decisiontree, file)

In [ ]:
# Load the Model back from file
with open(Pk2_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

Pickled_LR_Model

In [ ]:
# Use the Reloaded Model to 
# Calculate the accuracy score and predict target values

# Calculate the Score 
score = Pickled_LR_Model.score(X_val, y_val)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score))  

# Predict the Labels using the reloaded Model
Ypredict = Pickled_LR_Model.predict(X_val)  

Ypredict

In [ ]:
# GridSearchCV Tune the parameters 

params_dict = {
    'max_depth' : [3, 5, 7, 8, 11, 12],
    'min_samples_split' : [2, 3, 5, 9],
    'criterion' : ['gini', 'entropy']
}

grid = GridSearchCV(decisiontree, 
                   param_grid= params_dict,
                   cv = 10, 
                   verbose=1,
                   n_jobs=-1)
grid.fit(xtrain, ytrain)

In [ ]:
# Save the model to disk

In [ ]:
#DS_T = 'decisiontree.sav'

In [ ]:
#pickle.dump(decisiontree,open('DS_T','wb'))

In [ ]:
#loaded_model=pickle.load(open(Tree ,'rb'))
#result=loaded_model.score(xtest,ytest)
#print(result)

In [ ]:
grid.cv_results_

# Examine the best model

In [ ]:

print("Best params: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

# <a name='Rf' >Random Forest with GridSearchCV Tune the parameters</a>

In [ ]:


randomforest = RandomForestClassifier(n_estimators=100)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)
randomforest.fit(xtrain, ytrain)
print(randomforest.score(xtrain, ytrain))
print(randomforest.score(xtest, ytest))


In [ ]:
# Save the Modle to file in the current working directory

Pk3_Filename = "Pickle_RL_Model.pk3"  

with open(Pk3_Filename, 'wb') as file:  
    pickle.dump(randomforest, file)

In [ ]:
# Load the Model back from file
with open(Pk3_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

Pickled_LR_Model

In [ ]:
# Use the Reloaded Model to 
# Calculate the accuracy score and predict target values

# Calculate the Score 
score = Pickled_LR_Model.score(xtest, ytest)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score))  

# Predict the Labels using the reloaded Model
Ypredict = Pickled_LR_Model.predict(xtest)  

Ypredict

In [ ]:
# GridSearchCV Tune the parameters
params_dict = {
    'n_estimators': [10, 20, 40, 50, 70],
    'max_depth' : [3, 5, 7, 8, 11, 12],
    'criterion' : ['gini',  'entropy' ],
    'min_samples_split' : [  2 ,  3,  5 ,  9 ],
    'max_features' : [ 'auto' ,  'sqrt' ,   'log2' ]
}

grid = GridSearchCV(randomforest, 
                   param_grid= params_dict,
                   cv = 10, 
                   verbose=1,
                   n_jobs=-1)
grid.fit(xtrain, ytrain)

In [ ]:
grid.cv_results_

# Examine the best model

In [ ]:

print("Best params: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

In [ ]:
rf_best =grid.best_estimator_

# <a name='we' > Soft voting</a>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [ ]:
lr_model = linear_model.LogisticRegression()
knn_model = neighbors.KNeighborsClassifier()
rf_model = ensemble.RandomForestClassifier()

In [ ]:
model_names = ["lr_model","knn_best","rf_best"]

In [ ]:
for model_name in model_names:
    curr_model = eval(model_name)
    curr_model.fit(X_train, y_train)
    print(f'{model_name} Train score: {curr_model.score(X_train, y_train)}')
    print(f'{model_name} Test score: {curr_model.score(X_test, y_test)}')
    print('------')

In [ ]:
model_vars = [eval(n) for n in model_names]
model_list = list(zip(model_names, model_vars))

In [ ]:
model_list

# create voting classifier

In [ ]:

voting_classifer = VotingClassifier(estimators=model_list,
                                    voting='soft', #<-- sklearn calls this soft voting
                                    n_jobs=-1)
voting_classifer.fit(X_train, y_train)

# <a name='sc' >Stacking Classifier </a>

In [ ]:

stacked = StackingClassifier(
    classifiers=model_vars, meta_classifier=LogisticRegression(), use_probas=False)

In [ ]:
stacked.fit(X_train, y_train)

In [ ]:
y_pred = stacked.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
y_pred1 = stacked.predict(X_train)
accuracy_score(y_train, y_pred1)

# <a name='cm' >Confution Matrix </a>

In [ ]:
y_pred= knn_tune.predict(X_val)

In [ ]:
 precision_score(y_pred,y_val)

In [ ]:
 recall_score(y_pred,y_val)

In [ ]:
f1_score(y_val, y_pred)

In [ ]:
precision_recall_curve(y_pred,y_val)

In [ ]:
from sklearn.metrics import confusion_matrix
print("kNN confusion matrix: \n\n", confusion_matrix(y_val,knn_tune.predict(X_val)))

In [ ]:
precision_curve, recall_curve, threshold_curve = precision_recall_curve(y_val,knn_tune.predict_proba(X_val)[:,1] )
plt.figure(dpi=80)
plt.plot(threshold_curve, precision_curve[1:],label='precision')
plt.plot(threshold_curve, recall_curve[1:], label='recall')
plt.legend(loc='lower left')
plt.xlabel('Threshold (above this probability, label as fraud)');
plt.title('Precision and Recall Curves');

In [ ]:
# Print confusion matrix for kNN
knn_confusion = confusion_matrix(y_test,knn_tune.predict(X_test))
plt.figure(dpi=150)
sns.heatmap(knn_confusion, cmap=plt.cm.Blues, annot=True, square=True) 
plt.xlabel('Predicted species')
plt.ylabel('Actual species')
plt.title('KNN confusion matrix');

# <a name='jp' > XGboost </a>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
gbm = xgb.XGBClassifier( 
                        n_estimators=30000,
                        max_depth=4,
                        objective='binary:logistic', #new objective
                        learning_rate=.05, 
                        subsample=.8,
                        min_child_weight=3,
                        colsample_bytree=.8
                       )

eval_set=[(X_train,y_train),(X_test,y_test)]
fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='error', #new evaluation metric: classification error (could also use AUC, e.g.)
                    early_stopping_rounds=50,
                    verbose=False
                   )

accuracy_score(y_test, gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit)) 

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [ ]:
xgb.plot_importance(gbm, importance_type='gain')

In [ ]:
model = xgb.XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_comb = 70

random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs=-1, cv=10, verbose=3, random_state=42)

random_search.fit(X_train, y_train) 

In [ ]:
print('Best score:')
print(random_search.best_score_)

In [ ]:
print('Best estimator:')
print(random_search.best_estimator_)

In [ ]:
tuned_xgb = random_search.best_estimator_